# A股市场进行预测分析

In [17]:
#--coding:utf-8 ---
import csv
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib

name = ['现金评分', '经营能力评分', '获利能力评分', '财务结构评分', '偿债能力评分', '总评分:(3:3:2:2:1)']
years = np.array([2013, 2014, 2015, 2016, 2017], np.int64)
path_root = '..\\OutFile\\finally'
dirs = os.listdir(path_root)
dct = {}
S2 = []
for dir in dirs:
    a2 = 0
    file = path_root+'\\' + dir
    #print(file)
    file = csv.reader(open(file, 'r',encoding='utf-8'))
    ls = []
    data = np.zeros((37, 5), np.float32)
    for i, row in enumerate(file):
        for x in range(len(row)):
            if row[x] == '':
                row[x] = '0'
        if i==0:
            continue
        ls.append(row[0])
        for j in range(5):
            data[i-1, j] =np.float32(row[j+1])  
    def drowplt(data, years):
        plt.figure(figsize=(100, 100))
        #每支股票的全部的表
        for i in range(37):
            plt.subplot(7,6,i+1)
            plt.plot(years,data[i],'r',lw=2,label=ls[i])
            plt.title(ls[i])
        # 功能二获取每个股票柱状图
        plt.show()
    score = np.zeros((6, 5))
    hh = []
    for j in range(5):
        #求股票的气
        #求现金占比
        if data[0, j] >= 10:
            score[0, j] = score[0, j] + 7
        else:
            score[0, j] = score[0, j] + data[0, j]*7/ 25
        #求股票的应收款周转天数
        if data[14, j] <= 15:
            score [0, j] = score [0, j] + 2
        else:
            score [0, j] = score [0, j] + 2*15/data[14, j]
        #现金的三大比例
        i = 0
        if data[34, j] >= 100:      #现金流量比率
            i = 1
        else:
            i = (data[34, j]/100)
        if data[35, j] >= 100:      #现金允当比例
            i = i + 1
        else:
            i = (data[35, j]/100)+i
        if data[36, j] >= 10:       #现金在投资比例
            i = i + 1 
        else:
            i = i + (data[36, j]/100)
        score [0, j] = score[0, j] + i/3

        #经营能力
        #资产周转率
        if data[21, j] <1:
            score[1, j] = score[0, j] 
        else:
            score[1, j] = score[1, j] + 5
            if data[17, j] <= 30:     #存货周转天数
                score[1, j] = score[1, j] + 2.5
            else:
                score[1,j] = score[1, j] + 2.5*30/data[17, j]
            if data[14, j]<=90:      #应收款项周转天数
                score[1, j] = score[1, j] + 2.5
            else:
                score[1, j] = score[1, j] + 2.5*60/data[14, j]
        #获利能力
        #毛利率
        if data[25, j]>=32:    
            score[2, j] = score[2, j] + 2
        else:
            score[2, j] = score[2, j] + 2*data[25, j]/30
        #营业利益率
        if data[26, j] >= 10:
            score[2, j] = score[2, j] + 2
        else:
            score[2, j] = score[2, j] + 2*data[26, j]/10
        if data[29, j]>=60:    #经营安全边界
            score[2, j] = score[2, j] + 2
        else:
            score[2, j] = score[2, j] + 2*data[29, j]/60
        if data[27, j]>=2:     #净利率
            score[2, j] = score[2, j] + 1
        else:
            score[2, j] = score[2, j] + data[27, j]/2
        if data[30, j]>=1:      #EPS
            score[2, j] = score[2, j] + 1
        else:
            score[2, j] = score[2, j] + data[30, j]
        if data[23, j]>=20:     #ROE
            score[2, j] = score[2, j] + 1
        else:
            score[2, j] = score[2, j] + data[23, j]/20
        #财务结构
        #负债占资产比例
        if data[9, j]  >= 25 and data[9, j] <= 70:
            score[3, j] = score[3, j]+5
        elif data[9, j] < 25:
            score[3, j] = score[3, j] + 5*data[9, j]/25
        else:
            score[3, j] = score[3, j]+0
        #长期资金占不动产比率
        if data[10, j] >=100:
            score[3, j] = score[3, j] + 5
        else:
            score[3, j] = score[3, j] + 5*data[10, j]/100
        #偿债能力
        #流动比率
        if data[11, j] >= 300:
            score[4, j] = score[4, j] + 5
        else:
            score[4, j] = score[4, j] + 5*data[11, j]/300
        #速动比率
        if data[12, j] >= 150:
            score[4, j] = score[4, j] + 5
        else:
            score[4, j] = score[4, j] + 5*data[12, j]/100

        score[5, j] = (score[0, j] * 3 + score[1, j]* 3 + score[2, j]*2 + score[3, j] + score[4, j])
        a2 = a2+ score[5, j]
    s2 = (score[5,0] - a2/5)**2 + (score[5,1] - a2/5)**2+ (score[5,2] - a2/5)**2+ (score[5,3] - a2/5)**2+ (score[5,4] - a2/5)**2
    S2.append(s2)
#     s = pd.DataFrame(score,columns=years, index = name)
#     s.to_csv('..\\OutFile\\score\\'+dir)
    #将元组(2017的分数, 方差)写入字典中
    dct[dir] = (score[5, 4], round(float(s2),1))
print('完成')

完成


In [18]:
#对字典的内容对2017年的总评分进行排序
ls = list(dct.values())
ls.sort(reverse=True)
ls

[(95.232773310343433, 76.3),
 (94.536770900090531, 4.7),
 (94.135150477091486, 27.9),
 (93.24573659896852, 998.6),
 (93.226695268948887, 2.0),
 (93.134679902394595, 91.3),
 (92.899532286326092, 1345.0),
 (92.554208137994181, 1127.8),
 (92.474711488087976, 12.3),
 (92.440461730957026, 1875.3),
 (92.398318481445315, 426.5),
 (92.103731877009082, 87.1),
 (91.941042375564578, 7.1),
 (91.206580843528101, 105.8),
 (91.122101751963299, 960.3),
 (91.088130141099285, 168.4),
 (91.036995686690005, 8.5),
 (91.004720053672784, 1383.4),
 (90.986775932312014, 32.5),
 (90.966855311393729, 1931.8),
 (90.90434570789337, 33994.1),
 (90.894747595787038, 13.3),
 (90.832649063269287, 109.9),
 (90.783998592694587, 7.4),
 (90.763008516668705, 1.1),
 (90.457184145340932, 1611.8),
 (90.436411206460065, 749.9),
 (90.40018340545808, 10420.0),
 (90.273688713709532, 29.6),
 (90.219998902479801, 9.5),
 (90.044080728735693, 13.3),
 (89.897921901543924, 300.3),
 (89.856546013134079, 41.3),
 (89.623788331987896, 91.7)

In [19]:
dct

{'000001_平安银行.csv': (-118780004853.62895, 1.1202862158547582e+22),
 '000002_万  科Ａ.csv': (81.331209367116287, 971.7),
 '000004_国农科技.csv': (74.154614430467291, 205.0),
 '000005_世纪星源.csv': (30.212617512234811, 17240.6),
 '000006_深振业Ａ.csv': (84.156202130317695, 29.8),
 '000007_全新好.csv': (67.562046059918373, 1056.7),
 '000008_神州高铁.csv': (74.455427345752582, 192.4),
 '000009_中国宝安.csv': (69.338191751699839, 13.7),
 '000010_美丽生态.csv': (-152.13100230375477, 36141.9),
 '000011_深物业A.csv': (89.280559984842924, 41.4),
 '000012_南  玻Ａ.csv': (75.437811482460035, 1222.8),
 '000014_沙河股份.csv': (37.694309102197487, 1701.6),
 '000016_深康佳Ａ.csv': (80.016114262070502, 571.0),
 '000017_深中华A.csv': (67.778625659881669, 533.2),
 '000018_神州长城.csv': (58.169080562677962, 411.9),
 '000019_深深宝Ａ.csv': (32.831847337709348, 1492.4),
 '000020_深华发Ａ.csv': (71.297220915134005, 1777.7),
 '000021_深科技.csv': (76.696627898000514, 6.0),
 '000022_深赤湾Ａ.csv': (85.626514385952802, 1257.5),
 '000023_深天地Ａ.csv': (63.658113509345462, 28.6

In [36]:
llll=[]
for i in range(len(dirs)):
    for dir in dirs:
        if dct[dir] == ls[i] and ls[i][1]<= 50 and ls[i][1] != 0:
            s = dir.split('_')
            s1 = s[1].split('.')
            #print(s)
            llll.append([s[0], s1[0],round(float(ls[i][0]),1), ls[i][1]])

In [37]:
lll = pd.DataFrame(llll, columns=['股票代码', '股票名称', '评分', "方差"])

In [39]:
lll.to_csv('bzdgp.csv')